In [1]:
import pickle
import numpy

In [2]:
from pickle import load,dump

# Importing dataset

In [86]:
train=load(open('/Users/lav/Downloads/UPDATED_NLP_COURSE/06-Deep-Learning/train_qa.txt','rb'))

In [87]:
test=load(open('/Users/lav/Downloads/UPDATED_NLP_COURSE/06-Deep-Learning/test_qa.txt','rb'))

In [88]:
voc=train+test

In [91]:
from keras.preprocessing.text import Tokenizer

In [92]:
tokenizer=Tokenizer()

In [93]:
tokenizer.fit_on_texts([voccc for vocc in voc for voccc in vocc])

In [95]:
vocab_len=len(tokenizer.word_index)+1

In [96]:
l=[]
for story,question,answer in voc:
    le=len(story)
    l.append(le)

In [97]:
l.sort()

In [98]:
max_story_len=156

In [99]:
l=[]
for story,question,answer in voc:
    le=len(question)
    l.append(le)
l.sort()
max_question_len=l[-1]

In [100]:
max_question_len

6

## Vectorization using Keras

In [232]:
from keras.preprocessing.sequence import pad_sequences

In [180]:
que=[]
ans=[]
story=[]
for stories,questions,answers in train:
    story.append(stories)
    que.append(questions)
    ans.append(answers)

In [181]:
story_seq=tokenizer.texts_to_sequences(story)
que_seq=tokenizer.texts_to_sequences(que)
ans_seq=tokenizer.texts_to_sequences(ans)

In [182]:
input_train=pad_sequences(story_seq,maxlen=max_story_len)

In [183]:
query_train=pad_sequences(que_seq,maxlen=max_question_len)

In [184]:
answer_train=pad_sequences(ans_seq,maxlen=vocab_len)

In [185]:
que=[]
ans=[]
story=[]
for stories,questions,answers in test:
    story.append(stories)
    que.append(questions)
    ans.append(answers)
story_seq=tokenizer.texts_to_sequences(story)
que_seq=tokenizer.texts_to_sequences(que)
ans_seq=tokenizer.texts_to_sequences(ans)

input_test=pad_sequences(story_seq,maxlen=max_story_len)
query_test=pad_sequences(que_seq,maxlen=max_question_len)
answer_test=pad_sequences(ans_seq,maxlen=vocab_len)

In [186]:
answer_train.shape

(10000, 38)

In [189]:
from keras.models import Sequential,Model

In [190]:
from keras.layers import Embedding,Dense,Input,Activation,Permute,Dropout,add,dot,concatenate,LSTM

### Creating a placeholder

In [191]:
input_sequence=Input((max_story_len))
question=Input((max_question_len))

## Input Encoder m

In [192]:
input_encoder_m=Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len,output_dim=64))
input_encoder_m.add(Dropout(0.3))

## Input Encoder c

In [193]:
input_encoder_c=Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len,output_dim=max_question_len))
input_encoder_c.add((Dropout(0.3)))

## Question Encoder 

In [194]:
question_encoder=Sequential()
question_encoder.add(Embedding(input_dim=vocab_len,output_dim=64))
question_encoder.add(Dropout(0.3))

## Encode the sequence

In [195]:
input_encoded_m=input_encoder_m(input_sequence)
input_encoded_c=input_encoder_c(input_sequence)
question_encoded=question_encoder(question)

In [196]:
question_encoded.shape

TensorShape([None, 6, 64])

In [197]:
input_encoded_m.shape

TensorShape([None, 156, 64])

### Use dot product to comput the match between the input m vector and the u vector of the queries

In [198]:
match=dot([input_encoded_m,question_encoded],axes=(2,2))

In [199]:
match=Activation('softmax')(match)

In [200]:
match.shape

TensorShape([None, 156, 6])

In [201]:
input_encoded_c

<tf.Tensor 'sequential_6/dropout_6/cond/Identity:0' shape=(None, 156, 6) dtype=float32>

### Add match matrix to the 2nd input vector c

In [202]:
response=add([match,input_encoded_c])

In [203]:
question_encoded.shape

TensorShape([None, 6, 64])

In [204]:
response.shape

TensorShape([None, 156, 6])

In [205]:
response=Permute([2,1])(response)

In [206]:
response.shape

TensorShape([None, 6, 156])

### Concatenate the question vector and the response vector

In [207]:
answer=concatenate([response,question_encoded])

In [208]:
answer.shape

TensorShape([None, 6, 220])

### LSTM

In [209]:
answer=LSTM(32)(answer)

### Dropouts

In [210]:
answer=Dropout(0.5)(answer)
answer=Dense(vocab_len)(answer)

### Activation

In [211]:
answer=Activation('softmax')(answer)

In [212]:
model=Model([input_sequence,question],answer)

In [213]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [214]:
model.summary()

Model: "functional_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 156)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, None, 64)     2432        input_3[0][0]                    
__________________________________________________________________________________________________
sequential_7 (Sequential)       (None, None, 64)     2432        input_4[0][0]                    
______________________________________________________________________________________

### Fitting the model

In [215]:
model.fit([input_train,query_train],answer_train,batch_size=120,epochs=12,validation_data=([input_test,query_test],answer_test))

Epoch 1/12
84/84 [==============================] - 2s 24ms/step - loss: 22.5341 - accuracy: 0.9360 - val_loss: 0.1418 - val_accuracy: 1.0000
Epoch 2/12
84/84 [==============================] - 2s 19ms/step - loss: 0.2463 - accuracy: 1.0000 - val_loss: 0.0048 - val_accuracy: 1.0000
Epoch 3/12
84/84 [==============================] - 2s 18ms/step - loss: 0.0316 - accuracy: 1.0000 - val_loss: 2.3695e-04 - val_accuracy: 1.0000
Epoch 4/12
84/84 [==============================] - 2s 18ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 1.2638e-05 - val_accuracy: 1.0000
Epoch 5/12
84/84 [==============================] - 2s 19ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/12
84/84 [==============================] - 2s 20ms/step - loss: 2.2694e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/12
84/84 [==============================] - 2s 18ms/step - loss: 7.3635e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_a

### Saving the Model

In [219]:
filename = 'chatbot_120_epochs.h5'
model.save(filename)

### Evaluating on Given Test Set

In [222]:
model.load_weights(filename)
pred_results = model.predict(([input_test, query_test]))

In [224]:
test[0][0]

['Mary',
 'got',
 'the',
 'milk',
 'there',
 '.',
 'John',
 'moved',
 'to',
 'the',
 'bedroom',
 '.']

In [226]:
story =' '.join(word for word in test[0][0])
print(story)

Mary got the milk there . John moved to the bedroom .


In [227]:
query = ' '.join(word for word in test[0][1])
print(query)

Is John in the kitchen ?


In [228]:
print("True Test Answer from Data is:",test[0][2])

True Test Answer from Data is: no


In [230]:
import numpy as np

In [231]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  left
Probability of certainty was:  1.0
